In [2]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import optuna
torch.manual_seed(0)

# Prepare data

In [3]:
df = pd.read_csv('data/Monthly_Average_1950_2009_reservoir.csv')

In [4]:
batch_size = 32

tensor_x = torch.Tensor(df.values) # transform to torch tensor
dataset = TensorDataset(tensor_x) # create your datset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # create your dataloader

# Define a network

In [5]:
# linear, batch norm, relu block
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )

In [6]:
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, z_dim=2, im_dim=6, hidden_dim=8):
        super().__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
        )
    def forward(self, noise):
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):
        '''
        Returns:
            the sequential model
        '''
        return self.gen

In [7]:
def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim),
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    return torch.randn(n_samples,z_dim,device=device)

In [8]:
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
         nn.Linear(input_dim, output_dim),
         nn.LeakyReLU(0.2, inplace=True)
    )

In [9]:
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_dim: the dimension of the images, fitted for the dataset used, a scalar
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, im_dim=6, hidden_dim=8):
        super().__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):
        return self.disc(image)
    
    # Needed for grading
    def get_disc(self):
        '''
        Returns:
            the sequential model
        '''
        return self.disc

In [10]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    '''
    Return the loss of the discriminator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        real: a batch of real images
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        disc_loss: a torch scalar loss value for the current batch
    '''
    fake_noise = get_noise(num_images, z_dim, device=device)
    fake = gen(fake_noise)
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    return disc_loss

In [11]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    '''
    Return the loss of the generator given inputs.
    Parameters:
        gen: the generator model, which returns an image given z-dimensional noise
        disc: the discriminator model, which returns a single-dimensional prediction of real/fake
        criterion: the loss function, which should be used to compare 
               the discriminator's predictions to the ground truth reality of the images 
               (e.g. fake = 0, real = 1)
        num_images: the number of images the generator should produce, 
                which is also the length of the real images
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    Returns:
        gen_loss: a torch scalar loss value for the current batch
    '''
    fake_noise = get_noise(num_images, z_dim, device=device)
    fake = gen(fake_noise)
    disc_fake_pred = disc(fake)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss

# Optimize

In [12]:
# Set your parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_dim = 2
display_step = 500

# lr = 0.00001
device = 'cpu'

In [13]:
gen = Generator(z_dim).to(device)
disc = Discriminator().to(device) 
test_generator = False # Whether the generator should be tested

def objective(trial):
    
    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-6, 1e-2, log=True)
    gen_opt = getattr(torch.optim, optimizer_name)(gen.parameters(), lr=lr)
    disc_opt = getattr(torch.optim, optimizer_name)(disc.parameters(), lr=lr)
    
    # Train the model
    cur_step = 0
    mean_generator_loss = 0
    mean_discriminator_loss = 0
    for epoch in range(n_epochs):
        fake__ = []
        real__ = []
        for real in dataloader:
            cur_batch_size = len(real[0])
            real_ = real[0].to(device)        
            
            ### Update discriminator ###
            # Zero out the gradients before backpropagation
            disc_opt.zero_grad()

            # Calculate discriminator loss
            disc_loss = get_disc_loss(gen, disc, criterion, real_, cur_batch_size, z_dim, device)

            # Update gradients
            disc_loss.backward(retain_graph=True)

            # Update optimizer
            disc_opt.step()
            
            # For testing purposes, to keep track of the generator weights
            if test_generator:
                old_generator_weights = gen.gen[0][0].weight.detach().clone()

            # Update generator
            gen_opt.zero_grad()
            gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
            gen_loss.backward()
            gen_opt.step()

            # For testing purposes, to check that your code changes the generator weights
            if test_generator:
                try:
                    assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                    assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
                except:
                    error = True
                    print("Runtime tests have failed")
                    
            # Keep track of the average discriminator loss
            mean_discriminator_loss += disc_loss.item() / display_step

            # Keep track of the average generator loss
            mean_generator_loss += gen_loss.item() / display_step
            real__.append(real_.cpu().detach().numpy())
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake__.append(gen(fake_noise).cpu().detach().numpy())
            
            ### Visualization code ###
            if cur_step % display_step == 0 and cur_step > 0:
                print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
                fake_noise = get_noise(cur_batch_size, z_dim, device=device)
                fake = gen(fake_noise)
                # show_tensor_images(fake)
                # show_tensor_images(real)
                mean_generator_loss = 0
                mean_discriminator_loss = 0
            cur_step += 1

        # Evaluate the model
        fake_noise = get_noise(720, z_dim, device=device)
        fake = gen(fake_noise).cpu().detach().numpy()
        real = df.values
        
        # mean error (%)
        mean_error = (fake.mean(axis=0) - real.mean(axis=0)) / real.mean(axis=0) * 100
        mean_mape = np.abs(mean_error).mean()
        
        # Calculate empirical CDFs
        bins = np.linspace(0, 15, 61)   # Maximum 15 mm for now
        binsize = bins[1:] - bins[:-1]

        real_hist = np.apply_along_axis(lambda a: np.histogram(a, bins=bins, density=True)[0], 0, real)
        real_ecdf = np.cumsum(real_hist, axis=0) * binsize[:, np.newaxis]

        fake_hist = np.apply_along_axis(lambda a: np.histogram(a, bins=bins, density=True)[0], 0, fake)
        fake_ecdf = np.cumsum(fake_hist, axis=0) * binsize[:, np.newaxis]

        # Continuous ranked probability score (CRPS)
        crps = np.sum(np.abs((real_ecdf - fake_ecdf) * binsize[:, np.newaxis]), axis=0)
        crps_mean = crps.mean()
                          
    return crps_mean

In [14]:
study = optuna.create_study()
study.optimize(objective, n_trials=3)

[I 2023-03-01 11:58:18,587] A new study created in memory with name: no-name-7c0b0c55-e0e4-4b1c-9016-7bcdc7a966ec


Step 500: Generator loss: 0.7991444392204292, discriminator loss: 0.6456567057371135
Step 1000: Generator loss: 0.7593058142662048, discriminator loss: 0.5707634758949282
Step 1500: Generator loss: 0.6869591773748396, discriminator loss: 0.5444986629486083
Step 2000: Generator loss: 0.617787445425987, discriminator loss: 0.565177122950554
Step 2500: Generator loss: 0.594676935791969, discriminator loss: 0.5939955794811244
Step 3000: Generator loss: 0.6003148006200794, discriminator loss: 0.6201934701204301
Step 3500: Generator loss: 0.6137431548833846, discriminator loss: 0.6437540477514269
Step 4000: Generator loss: 0.6350214660167693, discriminator loss: 0.6603099629878997
Step 4500: Generator loss: 0.6651565856933592, discriminator loss: 0.6754032688140874


[I 2023-03-01 11:58:41,695] Trial 0 finished with value: 0.8307870370370388 and parameters: {'optimizer': 'Adam', 'lr': 1.7636164219259476e-05}. Best is trial 0 with value: 0.8307870370370388.


Step 500: Generator loss: 0.694727243304252, discriminator loss: 0.6873603744506825
Step 1000: Generator loss: 0.6967823829650871, discriminator loss: 0.6892094235420234
Step 1500: Generator loss: 0.7016329618692396, discriminator loss: 0.69172635936737
Step 2000: Generator loss: 0.7077283507585523, discriminator loss: 0.6936779215335852
Step 2500: Generator loss: 0.7145060927867894, discriminator loss: 0.6944568610191337
Step 3000: Generator loss: 0.7136772755384442, discriminator loss: 0.6953558348417288
Step 3500: Generator loss: 0.714115687012672, discriminator loss: 0.696205531358719
Step 4000: Generator loss: 0.7192076514959338, discriminator loss: 0.6967317454814909
Step 4500: Generator loss: 0.7210209337472916, discriminator loss: 0.6971840173006056


[I 2023-03-01 11:59:00,186] Trial 1 finished with value: 0.8909143518518544 and parameters: {'optimizer': 'SGD', 'lr': 0.00037779878181280074}. Best is trial 0 with value: 0.8307870370370388.


Step 500: Generator loss: 0.7670457631349562, discriminator loss: 0.7044006778001785
Step 1000: Generator loss: 0.8017038335800178, discriminator loss: 0.6971723412275315
Step 1500: Generator loss: 0.7301531696319588, discriminator loss: 0.6939190485477448
Step 2000: Generator loss: 0.7030692442655561, discriminator loss: 0.6933274445533759
Step 2500: Generator loss: 0.6957517606019975, discriminator loss: 0.693159306645394
Step 3000: Generator loss: 0.6981339417696003, discriminator loss: 0.6934807522296907
Step 3500: Generator loss: 0.6967850993871688, discriminator loss: 0.6939229289293289
Step 4000: Generator loss: 0.6921789439916611, discriminator loss: 0.6932142763137812
Step 4500: Generator loss: 0.695092509746551, discriminator loss: 0.69377520930767


[I 2023-03-01 11:59:21,853] Trial 2 finished with value: 0.7510582031357478 and parameters: {'optimizer': 'Adam', 'lr': 6.375358917290476e-05}. Best is trial 2 with value: 0.7510582031357478.


# Check results

In [16]:
pruned_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.7510582031357478
  Params: 
    optimizer: Adam
    lr: 6.375358917290476e-05
